In [1]:
import findspark
from pyspark.sql import SparkSession

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

23/07/17 14:15:08 WARN Utils: Your hostname, RNTDELL000700 resolves to a loopback address: 127.0.1.1; using 192.168.1.131 instead (on interface wlp0s20f3)
23/07/17 14:15:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/17 14:15:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sc = spark.sparkContext

### 1. Acciones en un RDD

Recordemos que las acciones son operaciones que activan los cálculos.\
Hasta que se encuentra una operación de acción el plan de ejecución dentro del programa Spark se crea en forma de DAG, por tanto, no se hará nada hasta que se realice una acción dentro del DAG. Podría haber varias transformaciones de todo tipo dentro del plan de ejecución, pero no sucede nada hasta que realicemos una acción.\
Cualquier tipo de variables que nosotros podamos tener en memoria acelerará grandemente nuestras aplicaciones de Spark, debido a que no se tendrá que ejecutar todo el DAG para reconstruir el RDD que querramos en ese momento.


#### Tipos de acciones
La acción activa todo el tag de las transformaciones construidas hasta ese momento, para que se materialicen ejecutando los bloques de código y las funciones. Existen dos tipos de operaciones de acción: 

   * **Las que ocurren en el driver:** suelen recopilar conteos, recuentos por tecla, etc. Cada una de estas acciones realiza algunos cálculos en el ejecutor remoto (cluster nodes) y devuelve los datos al driver.

   * **Las distribuidas:** Son los que se ejecutan en los nodos del cluster. Un ejemplo de tal acción distribuida es saveAsTextFile(); esta es la operación de acción más común debido a la naturaleza distribuida deseable de la misma operación.

### 1-1. Función reduce

Reduce aplica la función de reducción a todos los elementos del RDD y la envía al driver.\
Se aplica reduce con una función lambda, en el RDD se aplica la función de reducción a cada uno de los elementos del RDD y se envía la información al driver.\
Debemos tener en cuenta que al ser una acción, reduce automáticamente va a desencadenar el DAG y por lo tanto nos va a devolver un resultado.

In [3]:
rdd = sc.parallelize([2,4,6,8])

In [19]:
# Sumamos todos los elementos del RDD
rdd.reduce(lambda x,y: x + y)

20

In [5]:
# Creamos un RDD y multiplicamos sus elementos entre sí
rdd1 = sc.parallelize([1,2,3,4])

rdd1.reduce(lambda x,y: x * y)

24

### 1-2. Función count

Cuenta el número de elementos del RDD y lo envía al controlador.\
El driver le pide a cada ejecutor que cuente la cantidad de elementos en la partición que está siendo manejada por la tarea y luego suma los recuentos de todas las tareas juntas en el nivel del driver.

In [6]:
rdd.count()

4

In [7]:
rdd1 = sc.parallelize([item for item in range(10)])
rdd1.count()

10

### 1-3. Función collect

Recopila todos los elementos del RDD y lo envía al driver. El driver extrae todos los elementos del RDD de todas las particiones.

In [8]:
rdd_frase = sc.parallelize('Hola Apache Spark!'.split(' '))
rdd_frase.collect()

['Hola', 'Apache', 'Spark!']

In [9]:
rdd2 = sc.parallelize([(item, item ** 2) for item in range(20)])
rdd2.collect()

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81),
 (10, 100),
 (11, 121),
 (12, 144),
 (13, 169),
 (14, 196),
 (15, 225),
 (16, 256),
 (17, 289),
 (18, 324),
 (19, 361)]

### 1-4. Funciones take, max y saveAsTextFile

In [10]:
rdd_frase2 = sc.parallelize('La programación es bella'.split(' '))

In [11]:
# take recibe un entero N y nos devuelve los primero N elementos que encuentre dentro del RDD
rdd_frase2.take(2)

['La', 'programación']

In [12]:
rdd_frase2.take(4)

['La', 'programación', 'es', 'bella']

In [14]:
# max nos va a devolver el máximo de los números que encuentren un RD que esté construido a partir de números.
rdd3 = sc.parallelize([item/(item + 1) for item in range(10)])
rdd3.max()

0.9

In [15]:
rdd3.collect()

[0.0,
 0.5,
 0.6666666666666666,
 0.75,
 0.8,
 0.8333333333333334,
 0.8571428571428571,
 0.875,
 0.8888888888888888,
 0.9]

In [16]:
# saveAsTextFile guarda los elementos de un RDD en un fichero de texto
# Esto nos lo va a guardar en un montón de ficheros
rdd_frase2.saveAsTextFile('./rdd')

In [18]:
# Para que lo guarde en un solo fichero, combinamos saveAsTexFile con coalesce
rdd_frase2.coalesce(1).saveAsTextFile('./rdd1')

>\
>Si los ficheros no son muy grandes, comprimirlos en una sola partición puede facilitar las acciones que hagamos sobre ellos
>
><br>